### Custom MBA courses RAG project
---

In [ ]:
# Go one level up in the directories hierarchy to access src directory and codes
import sys
import os
# Add project root to Python path
project_root = os.path.abspath("..")  # go one level up from notebooks/
sys.path.append(project_root)

### Test the RagSystem class defined

In [2]:
# Let's test our RAG module defined in class to see how well it is refactored
from core.src.rag.rag_workflow import RagChatWorkflow

import nest_asyncio
nest_asyncio.apply()

rag_chat = RagChatWorkflow()

c:\Users\Sanzhar\workspace\github.com\kaydarovv\ai_agent_learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2026-01-05 15:09:53,782 - WARNING - Ignoring wrong pointing object 9 0 (offset 0)
2026-01-05 15:09:53,783 - WARNING - Ignoring wrong pointing object 20 0 (offset 0)
2026-01-05 15:09:53,783 - WARNING - Ignoring wrong pointing object 22 0 (offset 0)
2026-01-05 15:09:53,784 - WARNING - Ignoring wrong pointing object 30 0 (offset 0)
2026-01-05 15:09:53,785 - WARNING - Ignoring wrong pointing object 36 0 (offset 0)
2026-01-05 15:09:53,786 - WARNING - Ignoring wrong pointing object 49 0 (offset 0)
2026-01-05 15:09:53,786 - WARNING - Ignoring wrong pointing object 55 0 (offset 0)
2026-01-05 15:09:53,788 - WARNING - Ignoring wrong pointing object 82 0 (offset 0)
2026-01-05 15:09:53,788 - WARNING - Ignori

In [3]:
user_name = "SomeUser"
user_id = "01"
user_query = "What are the differences between Financial and Managerial accounting?"

rag_response = await rag_chat.run(
    user_query = user_query,
    user_name = user_name, 
    user_id = user_id
)
print(rag_response)

2026-01-05 15:11:27,049 - INFO - AFC is enabled with max remote calls: 10.
2026-01-05 15:11:28,451 - INFO - Selecting retriever 0: This choice describes financial accounting, focusing on reading, analyzing, and interpreting financial accounting information for external stakeholders. It details the financial statements and the principles governing their preparation, which are key aspects of financial accounting. While it doesn't explicitly contrast with managerial accounting, it provides a clear definition of one side of the comparison..
2026-01-05 15:11:28,479 - INFO - AFC is enabled with max remote calls: 10.
2026-01-05 15:11:29,378 - INFO - AFC is enabled with max remote calls: 10.


SomeUser, Financial accounting is an external accounting system prepared under external rules like IFRS and US-GAAP, audited by external firms, and intended for external decision-makers such as shareholders and creditors, measuring business activities in monetary units. Managerial accounting, conversely, is an internal accounting system prepared under internal guidelines, is not audited, and is intended for internal managers, utilizing both monetary and non-monetary units.


In [4]:
rag_response = await rag_chat.run(
    user_query = "Summarize in 2 sentences what we dicussed so far",
    user_name = user_name, 
    user_id = user_id
)
print(rag_response)

2026-01-05 15:11:36,681 - INFO - AFC is enabled with max remote calls: 10.
2026-01-05 15:11:38,236 - INFO - Selecting retriever 0: This course focuses on reading, analyzing, and interpreting financial accounting information to make informed business decisions, covering financial statements, accounting mechanics, and analysis methods..
2026-01-05 15:11:38,238 - INFO - Selecting retriever 1: This course provides an analytical study of financial flows and decision-making, covering foundational theories, capital markets, and valuation principles..
2026-01-05 15:11:38,239 - INFO - Selecting retriever 5: This course focuses on formulating and implementing business and corporate strategy to achieve superior performance, integrating different perspectives for complex management situations..
2026-01-05 15:11:38,310 - INFO - AFC is enabled with max remote calls: 10.
2026-01-05 15:11:38 | WARNING  | core.src.rag.rag_workflow:_is_retrieval_relevant:122 - Among retrieved nodes, no nodes contain rel

SomeUser, we have discussed the fundamental differences between financial and managerial accounting, highlighting their distinct purposes, reporting standards, audiences, and the types of information they convey. Financial accounting serves external stakeholders with audited reports adhering to GAAP or IFRS, while managerial accounting supports internal decision-making with unaudited, often non-monetary data.
